# Implementing efficient recorders

In [1]:
import fwdpy11
import fwdpy11.model_params
import fwdpy11.fitness
import fwdpy11.wright_fisher

def evolve(args):
    """
    Evolve function takes an initial
    population size,
    RNG seed, and the type name of a recorder as
    arguments.
    """
    N,seed,recorderType,theta,rho,simlen=args
    pop = fwdpy11.SlocusPop(N)
    rng=fwdpy11.GSLrng(seed)
    nrate=theta/float(4*N)
    recrate=rho/float(4*N)
    params=fwdpy11.model_params.SlocusParams(
        nregions=[fwdpy11.Region(0,1,1)],
        sregions=[fwdpy11.ExpS(0,1,1,-0.1,1.0)],
        recregions=[fwdpy11.Region(0,1,1)],
        gvalue=fwdpy11.fitness.SlocusAdditive(2.0),
        demography=np.array([N]*simlen,dtype=np.uint32),
        rates=(nrate,5e-3,recrate))
    recorder = None
    if recorderType is not None:
        recorder=recorderType(simlen)
    fwdpy11.wright_fisher.evolve(rng,pop,params,recorder)
    return recorder.data

In [2]:
import numpy as np

class PyRecorder(object):
    def __init__(self,simlen):
        self.i = 0
        self.data=np.array([(-1,np.nan,np.nan,np.nan)]*simlen,
                          dtype=[('generation',np.uint32),
                                ('wbar',np.float),
                                ('s1',np.float),
                                ('s2',np.float)])
    def __call__(self,pop):
        fitness = np.zeros(pop.N,dtype=np.float)
        nmuts1 = np.zeros(pop.N,dtype=np.float)
        nmuts2 = np.zeros(pop.N,dtype=np.float)
        j=0
        for dip in pop.diploids:
            fitness[j]=dip.w
            n1 = 0.
            for j in pop.gametes[dip.first].smutations:
                n1 += pop.mutations[j].s
            n2 = 0.
            for j in pop.gametes[dip.second].smutations:
                n2 += pop.mutations[j].s
            nmuts1[j]=n1
            nmuts2[j]=n2
            j+=1
        self.data[self.i] = (pop.generation,fitness.mean(),nmuts1.mean(),nmuts2.mean())
        self.i += 1
        
class PyRecorder2(object):
    def __init__(self,simlen):
        self.i = 0
        self.data=np.array([(-1,np.nan,np.nan,np.nan)]*simlen,
                          dtype=[('generation',np.uint32),
                                ('wbar',np.float),
                                ('s1',np.float),
                                ('s2',np.float)])
    def __call__(self,pop):
        wbar = np.array(pop.diploids.trait_array())['w'].mean()
        esizes = np.array(pop.mutations.array())['s']
        #nmuts1 = np.zeros(pop.N,dtype=np.float)
        #nmuts2 = np.zeros(pop.N,dtype=np.float)
        j=0
        nm1=0.
        nm2=0.
        for dip in pop.diploids:
            nm1 += esizes[np.array(pop.gametes[dip.first].smutations)].sum()
            nm2 += esizes[np.array(pop.gametes[dip.second].smutations)].sum()
            j+=1
            
        self.data[self.i] = (pop.generation,wbar,nm1/float(pop.N),nm2/float(pop.N))
        self.i += 1

In [3]:
%load_ext Cython

In [4]:
%%cython --cplus --compile-args=-std=c++11
from libcpp.vector cimport vector
from libc.stdint cimport uint32_t
from cython.view cimport array as cvarray
import numpy as np
cimport numpy as np
cimport cython

cdef struct my_data:
    uint32_t generation
    double wbar,s1,s2

@cython.boundscheck(False)
cdef inline double sum_e(uint32_t[:] keys,double[:] esizes) nogil:
    cdef size_t i = 0, l = keys.shape[0]
    cdef double t = 0.
    while i < l:
        t += esizes[i]
        i+=1

cdef class CyRecorder(object):
    cdef my_data d
    cdef readonly vector[my_data] data
    
    def __cinit__(self,uint32_t simlen):
        self.data = vector[my_data]()
        self.data.reserve(simlen)
        
    def __call__(self,pop):
        wbar = np.array(pop.diploids.trait_array())['w'].mean()
        cdef np.ndarray[double,ndim=1] esizes = np.array(pop.mutations.array())['s']
        cdef double[:] esizes_view = esizes
        cdef double g1,g2,t1=0.,t2=0.
        for dip in pop.diploids:
            g1 = sum_e(pop.gametes[dip.first].mutations,esizes_view)
            g2 = sum_e(pop.gametes[dip.second].mutations,esizes_view)
            t1 += g1
            t1 += g2
        self.d.generation = pop.generation
        self.d.wbar = wbar
        self.d.s1 = t1/<double>(pop.N)
        self.d.s2 = t2/<double>(pop.N)
        self.data.push_back(self.d)

In [5]:
import cppimport
cppimport.set_quiet(True)
import os
sampler = cppimport.imp("sampler")

In [9]:
%timeit x=evolve((1000,42,PyRecorder,100.,100.,500))

52.8 s ± 604 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit x=evolve((1000,42,PyRecorder2,100.,100.,500))

35.1 s ± 1.36 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit x=evolve((1000,42,CyRecorder,100.,100.,500))

27.1 s ± 519 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit x=evolve((1000,42,sampler.cppRecorder,100.,100.,500))

238 ms ± 6.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit x=evolve((10000,42,sampler.cppRecorder,100.,100.,500))

1.8 s ± 57.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit x=evolve((1000,42,sampler.cppRecorder,1000.,1000.,500))

599 ms ± 24.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit x=evolve((10000,42,sampler.cppRecorder,1000.,1000.,500))

2.41 s ± 42.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
